## Collect experimental data and process csv

In [ ]:
from pathlib import Path
import json
from natsort import natsorted
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from neuroprocessing.scripts.parse_csv import parse_csv, process_data


## Processing

In [ ]:
def identify_trial_save_paths(exp_dir:str, trial_dir:str, params:dict) -> tuple:
    """Identify the paths to the raw and processed tiff stacks for a single trial
    
    Inputs:
        exp_dir: str
            Name of the experiment directory e.g. "2024-03-06"
        trial_dir: str
            Name of the trial directory e.g. "Zyla_30min_LHL_27mMhistinj_1pt75pctISO_1
        params: dict
            Run parameters
    Returns:
        tuple
            (Path to the raw tiff stack, Path to the processed tiff stack)
    """
    load_from_s3 = params["load_from_s3"]
    save_to_s3 = params["save_to_s3"]
    if load_from_s3:
        print("Loading raw stack from S3")
    else:
        print("Loading raw stack from local filesystem")

    trial_path = params["s3fs_toplvl_path"] if load_from_s3 else params["local_toplvl_path"]
    save_path = params["s3fs_toplvl_path"] if save_to_s3 else params["local_toplvl_path"]

    trial_path = Path(trial_path) / exp_dir / trial_dir
    save_path = Path(save_path) / exp_dir / trial_dir
    return (trial_path, save_path)

def _get_sync_info(sync_csv_path):
    """Get dict of sync info (stim time, etc) from the sync csv file
    """
    df_daq = parse_csv(sync_csv_path)

    df_frames = process_data(
        df_daq,
        col_camera="camera",
        col_stim="button"
    )

    stim_onset_frame = int(df_frames.loc[df_frames["stimulated"], "frame"].iloc[0])
    df_frames.set_index("frame", inplace=True)
    # print info
    print(f"Stimulus onset frame: {stim_onset_frame}")
    print(f"Stimulus onset time (s): {df_frames.loc[stim_onset_frame, 'time']}")
    stim_duration_frames = sum(df_frames['stimulated'])
    frame_time_s = df_frames.loc[stim_onset_frame, 'frametime']
    framerate_hz = 1/frame_time_s

    sync_info = {
        "stim_onset_frame": stim_onset_frame,
        "stim_duration_frames": stim_duration_frames,
        "frame_time_s": frame_time_s,
        "framerate_hz": framerate_hz
    }
    print(f"Stimulus duration (frames): {stim_duration_frames}")
    print(f"Stimulus duration (s): {stim_duration_frames/framerate_hz}")
    print(f"Frame duration (ms): {frame_time_s*1000}")
    print(f"Framerate (Hz): %.2f" % (framerate_hz))
    return sync_info

def process_trial(exp_dir:str, trial_dir:str, params:dict):
    """Process single trial
    
    Process a single imaging trial and save the processed tiff stack to the same directory as the original tiff stack.

    An imaging trial can be split into 2+ videos due to tiff file size limits.

    Stimulus is assumed to be in the first video.
    
    Inputs:
        exp_dir: str 
            Experiment dir e.g. "2024-03-06"
        trial_dir: str 
            Trial dir e.g. "Zyla_15min_LHL_salineinj_1pt75pctISO_1"
    """

    trial_path, save_path = identify_trial_save_paths(exp_dir, trial_dir, params)

    # load sync json if exists
    if (save_path / "sync_info.json").exists():
        with open(save_path / "sync_info.json", "r") as f:
            sync_info = json.load(f)
    else:
        sync_info = {}
        sync_info["stim_onset_frame"] = 3000
        print(f"Warning: No sync_info.json found. Using default value of {sync_info['stim_onset_frame']} for stimulus onset time.")
    
    # load processed tiff stack
    fp_processed_tif = save_path / (params["preprocess_prefix"] + trial_dir + ".tif")
    if not fp_processed_tif.exists():
        raise FileNotFoundError(f"Error: No preprocessed file exisits: {fp_processed_tif}")

    stack = io.imread(fp_processed_tif)
    # crop image to remove the black border that occurs due to motion registration
    crop_px = params["crop_px"]
    stack = stack[:, crop_px:-crop_px, crop_px:-crop_px]

    # only process frames after stimulus onset
    stack = stack[int((sync_info["stim_onset_frame"]+50)/params['downsample_factor']):, :, :]

    # find bottom X% of pixels in image
    bottom10 = np.percentile(stack, params['bottom_percentile'], axis=(1,2), keepdims=True).astype(np.uint16)

    # subtract bottom X% from all pixels
    stack -= bottom10
    stack -= stack.min(axis=0, keepdims=True)
    stack[stack > 30000] = 0

    # save tiff stack with the name of the first tiff in the stack
    io.imsave(save_path / (params["process_prefix"] + trial_dir + '.tif'), stack)


## Preprocessing

In [ ]:
import json, tqdm
from skimage.measure import block_reduce
from neuroprocessing.align import StackAligner

def preprocess_trial(exp_dir:str, trial_dir:str, params:dict):
    """Preprocessing of a single imaging trial

    A single trial may have multiple videos due to tiff file size limits.
        1. Downsample the tiff stack
        2. Motion correction
    
        Notes: 
            - Tiff stacks with MMStack in the name will be processed and concatenated if there are multiple.
        Usage:
            preprocess_trial("2024-03-06", "Zyla_15min_LHL_salineinj_1pt75pctISO_1", params)
        
    Inputs:
        exp_dir: str 
            Experiment dir e.g. "2024-03-06"
        trial_dir: str 
            Trial dir e.g. "Zyla_15min_LHL_salineinj_1pt75pctISO_1"
    """
    
    trial_path, save_path = identify_trial_save_paths(exp_dir, trial_dir, params)

    fp_tifs = natsorted(Path(trial_path).glob("*MMStack*.tif"))
    fp_csv = natsorted(Path(trial_path).glob("*.csv"))
    
    # if csv sync not found, ignore (**TODO** handle this)
    if len(fp_csv) == 0:
        print(f"No sync file found in {trial_path}")
    else:
        sync_info = _get_sync_info(fp_csv[0])
        # save sync info as json
        with open(save_path / "sync_info.json", "w") as f:
            json.dump(sync_info, f)
    
    stack_list = []
    for fp_tif in fp_tifs:
        stack_list.append(io.imread(fp_tif))

    # Downsample image
    stack_downsampled = block_reduce(np.concatenate(stack_list, axis=0), block_size=(params['downsample_factor'], 1, 1), func=np.mean)

    # temporarily save downsampled stack
    # io.imsave(save_path / ("downsampled_" + trial_dir + ".tif"), stack_downsampled.astype(np.uint16))
    del(stack_list)

    aligner = StackAligner(
            stack=stack_downsampled,
            target_num_features = params["aligner_target_num_features"]
        )
    aligner.align()
    
    # save aligned stack
    stack_aligned = aligner.stack_aligned

    io.imsave(save_path / (params["preprocess_prefix"] + trial_dir + ".tif"), stack_aligned.astype(np.uint16))


## Run

In [ ]:
params = {
    "downsample_factor": 8,
    "aligner_target_num_features": 700,
    "preprocess_prefix": "aligned_downsampled_",
    "process_prefix": 'processed_',
    "s3fs_toplvl_path": "/Users/ilya_arcadia/arcadia-neuroimaging-pruritogens/Videos",
    "local_toplvl_path": "/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections/",
    "load_from_s3": True,
    "save_to_s3": False,
    'crop_px' : 20,
    'bottom_percentile' : 10
}

def preprocess_and_process_trial(exp_dir:str, trial_dir:str, params:dict):
    """Preprocess and process a single trial
    
    Inputs:
        exp_dir: str 
            Experiment dir e.g. "2024-03-06"
        trial_dir: str 
            Trial dir e.g. "Zyla_15min_LHL_salineinj_1pt75pctISO_1"
    """
    preprocess_trial(exp_dir, trial_dir, params)
    process_trial(exp_dir, trial_dir, params)

preprocess_and_process_trial("2024-03-06", "Zyla_15min_LHL_salineinj_1pt75pctISO_1", params)
preprocess_and_process_trial("2024-03-06", "Zyla_15min_RHL_salineinj_1pt25pctISO_1", params)
preprocess_and_process_trial("2024-03-06", "Zyla_30min_LHL_27mMhistinj_1pt75pctISO_1", params)
preprocess_and_process_trial("2024-03-06", "Zyla_30min_RHL_27mMhistinj_1pt25pctISO_1", params)